In [1]:
import yfinance as yf
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
nse = "TCS.NS"

In [3]:
start_date=datetime.datetime(2024,4,1)
end_date=datetime.datetime(2025,4,30)

In [4]:
stock = yf.download(nse, start_date,end_date)
stock.index = pd.to_datetime(stock.index)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
print(stock.head())


Price             Close         High          Low         Open   Volume
Ticker           TCS.NS       TCS.NS       TCS.NS       TCS.NS   TCS.NS
Date                                                                   
2024-04-01  3809.312256  3825.408331  3781.399554  3790.784756  1569676
2024-04-02  3777.266113  3802.601601  3766.762313  3783.295998  2142666
2024-04-03  3839.024414  3871.799965  3745.365905  3745.365905  3973090
2024-04-04  3893.488281  3918.142780  3827.304866  3866.839729  3394637
2024-04-05  3870.098145  3892.224106  3851.862462  3868.882432  1636819


In [6]:
df = yf.download(
    tickers=nse,
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",

)
df

[*********************100%***********************]  1 of 1 completed


Ticker           TCS.NS                                                
Price              Open         High          Low        Close   Volume
Date                                                                   
2024-04-01  3790.784756  3825.408331  3781.399554  3809.312256  1569676
2024-04-02  3783.295998  3802.601601  3766.762313  3777.266113  2142666
2024-04-03  3745.365905  3871.799965  3745.365905  3839.024414  3973090
2024-04-04  3866.839729  3918.142780  3827.304866  3893.488281  3394637
2024-04-05  3868.882432  3892.224106  3851.862462  3870.098145  1636819
...                 ...          ...          ...          ...      ...
2025-04-23  3365.000000  3420.899902  3337.300049  3413.000000  3543923
2025-04-24  3401.100098  3412.500000  3387.699951  3401.600098  2252175
2025-04-25  3420.000000  3477.800049  3405.000000  3448.000000  2742991
2025-04-28  3435.000000  3457.800049  3405.000000  3443.500000  1593296
2025-04-29  3455.000000  3495.000000  3430.399902  3472.500000  1578332

[266 rows x 5 columns]

In [7]:
df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 266 entries, 2024-04-01 to 2025-04-29
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   (TCS.NS, Open)    266 non-null    float64
 1   (TCS.NS, High)    266 non-null    float64
 2   (TCS.NS, Low)     266 non-null    float64
 3   (TCS.NS, Close)   266 non-null    float64
 4   (TCS.NS, Volume)  266 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 12.5 KB


In [22]:


# Create the figure
fig = go.Figure(data=[

    # Candlestick chart
    go.Candlestick(
        x=df.index,
        open=df[nse]['Open'],
        high=df[nse]['High'],
        low=df[nse]['Low'],
        close=df[nse]['Close'],
        name='Market Data'
    ),

    # Highlight a significant drop with a red marker and label
    go.Scatter(
        x=['2024-06-05'],
        y=[3881],
        mode='markers+text',
        marker=dict(size=12, color='red', symbol='circle'),
        text=["Significant Drop"],
        textposition="top center",
        name="Noteworthy Event"
    ),

    # Vertical line for a potential reversal point
    go.Scatter(
        x=['2025-03-02', '2025-03-02'],
        y=[df[nse]['Low'].min() - 10, df[nse]['High'].max() + 10],
        mode='lines',
        line=dict(color='green', width=4, dash='dot'),
        name='Potential Reversal Point'
    )
])

# Update layout for styling and presentation
fig.update_layout(
    title={
        'text': f"Financial Performance: {nse}",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Trading Date',
    yaxis_title='Value',
    hovermode="x unified",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)

# Show the figure
fig.show()


In [24]:


# Calculate returns
df[(nse, 'Daily Return')] = df[nse]['Close'].pct_change()
df[(nse, 'Log Return')] = np.log(df[nse]['Close'] / df[nse]['Close'].shift(1))

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Simple Return", "Log Return")
)

# Add daily return line plot
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df[(nse, 'Daily Return')],
        mode='lines',
        name='Daily Return'
    ),
    row=1, col=1
)

# Add log return line plot
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df[(nse, 'Log Return')],
        mode='lines',
        name='Log Return'
    ),
    row=1, col=2
)

# Update layout for appearance
fig.update_layout(
    title_text="Problem 2",
    title_x=0.5,
    height=500,
    showlegend=False,
    template="plotly_dark"
)

# Display the plot
fig.show()


Volatility Estimation

In [25]:


# Calculate 14-day rolling standard deviation of daily returns
df.loc[:, (nse, 'Rolling Std Dev')] = df[nse]['Daily Return'].rolling(window=14).std()

# --- Plot 1: Rolling Standard Deviation Line Chart ---
fig1 = px.line(
    df,
    x=df.index,
    y=df[nse]['Rolling Std Dev'],
    title='Rolling 14-Day Standard Deviation'
)
fig1.update_layout(
    xaxis_title='Date',
    yaxis_title='Standard Deviation',
    template='plotly_dark'
)
fig1.show()

# --- Plot 2: Candlestick Chart for Price ---
fig2 = go.Figure(data=[
    go.Candlestick(
        x=df.index,
        open=df[nse]['Open'],
        high=df[nse]['High'],
        low=df[nse]['Low'],
        close=df[nse]['Close'],
        name='Price'
    )
])
fig2.update_layout(
    title=f'Candlestick Chart - {nse}',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark'
)
fig2.show()


Simple vs Log Returns

**Simple returns** are calculated as the percentage change in price:  
R_t = (Pt-Pt-1)/Pt-1

**Log returns** are calculated using natural logarithms:  
Rt=ln(Pt/Pt-1)

For daily data with small price changes, both returns look very similar. This is because when changes are small, ln(1 + x) approx x. However, during periods of large price movements (e.g., earnings surprises or market crashes), log returns tend to differ more noticeably due to their compounding nature.

Log returns are **time additive**, meaning that the return over multiple periods can be obtained by summing the individual log returns — a useful property in financial modeling and portfolio optimization.

In contrast, simple returns are easier to interpret intuitively as percentage gains or losses. Both are valid tools, and the choice depends on the context — log returns are often used in quantitative models, while simple returns are more common in performance reporting.


**Problem Statement - 3**

In [11]:
from scipy.stats import binom

In [14]:
# Determine the direction of daily price movement
df.loc[:, (nse, 'Market_Direction')] = df[nse]['Daily Return'].apply(lambda x: 'UP' if x > 0 else 'DOWN')

# Calculate the total number of valid trading days
total_trading_periods = df[nse]['Market_Direction'].count()

# Count the number of 'UP' days
favorable_days = (df[nse]['Market_Direction'] == 'UP').sum()

# Calculate the probability of an 'UP' day
prob_up_movement = favorable_days / total_trading_periods

print(f'P(UP) = {prob_up_movement:.3f}')
P_UP = prob_up_movement

P(UP) = 0.459


In [15]:
# Calculate theoretical probabilities based on P(UP) = 0.6
theoretical_exact_6_up = binom.pmf(k=6, n=10, p=0.6)
theoretical_at_least_8_up = 1 - binom.cdf(k=7, n=10, p=0.6)

# Calculate actual probabilities using the derived P_UP from stock data
actual_exact_6_up = binom.pmf(k=6, n=10, p=P_UP)
actual_at_least_8_up = 1 - binom.cdf(k=7, n=10, p=P_UP)

print(f"Theoretical Probability of exactly 6 up days in 10: {theoretical_exact_6_up:.3f}")
print(f"Theoretical Probability of at least 8 up days in 10: {theoretical_at_least_8_up:.3f}")
print(f"Actual Probability of exactly 6 up days in 10: {actual_exact_6_up:.3f}")
print(f"Actual Probability of at least 8 up days in 10: {actual_at_least_8_up:.3f}")

Theoretical Probability of exactly 6 up days in 10: 0.251
Theoretical Probability of at least 8 up days in 10: 0.167
Actual Probability of exactly 6 up days in 10: 0.168
Actual Probability of at least 8 up days in 10: 0.031


In [16]:
coin_flips=np.random.binomial(n=10, p=0.6, size=1000)

In [17]:
print(coin_flips)

[ 6  9  6  8  9  7  6  5  6  4 10  7  5  9  5  4  5  6  6  7  6  5  6  8
  6  4  6  7  4  6  5  3  8  4  8  8  6  9  4  5  6  4  6  6  6  8  8  6
  6  8  6  7  5  9  4  2  5  6  4  7  6  7  7  5  7  4  7  4  6  5  7  5
  5  7  8  8  3  6  3  7  5  6  3  3  7  4  7  7  6  5  8  6  5  6  7  6
  7  3  6  7  6  5  8  6  6  5  7  7  6  5  5  4  8  4  7  6  6  8  6  4
  7  7  5  5  6  7  3  9  8  6  8  8  6  7  7  6  9  6  5  3 10  5  7  7
  5  3  4  8  6  8  7  6  7  9  5  7  7  7  4  5  7  6  7  6  4  7  7  7
  6  4  6  8  9  6  4  6  5  4  6  7  7  6  8  8  9  4  7  9  3  8  5  6
  5  6  8  7  6  6  6  3  6  7  7  5  4  8  6  7  9  5  8  6  8  8  7  5
  8  8  5  5  4  7  8  4  6  6  5  4  5  6  3  7  8  7  5  6  5  9  3  6
  8  5  5  6  9  7  6  6  6  5  5  8  6  6  8  5  7 10  6  7  7  8  6  6
  6  7  2  5  7  6  5  7  7  2  8  8  8  6  7  7  5  7  7  8  7  7  5  4
  8  6  6  6  5  7  5  6  8  4  8  4  6  4  4  9  6  5  7  8  6  7  6  6
  4  5  7  5  7  7  4  5  7  5  5  6  9  7  5  5  8

In [18]:
# Calculate the simulated probability of observing exactly 6 successful outcomes
simulated_frequency_of_6 = np.mean(coin_flips == 6)

# Calculate the simulated probability of observing 8 or more successful outcomes
simulated_frequency_of_8_or_more = np.mean(coin_flips >= 8)

print(simulated_frequency_of_6)
print(simulated_frequency_of_8_or_more)

0.248
0.191


**Part 2: The Paradox**

In [19]:
# Calculate the mean daily return for 'UP' market days
average_upward_return = df[nse].loc[df[nse]['Market_Direction'] == 'UP', 'Daily Return'].mean()

# Calculate the mean daily return for 'DOWN' market days
average_downward_return = df[nse].loc[df[nse]['Market_Direction'] == 'DOWN', 'Daily Return'].mean()

print(f'Mean return on upward days: {average_upward_return}')
print(f'Mean return on downward days: {average_downward_return}')

Mean return on upward days: 0.010059419670158875
Mean return on downward days: -0.009064935266109662


In [20]:
results = []
data = df[nse][['Daily Return', 'Market_Direction']].copy()
for i in range(len(data) - 9):
    window = data.iloc[i:i+10]#rolling 10 day window

    up_returns = window[window['Market_Direction'] == 'UP']['Daily Return']
    down_returns = window[window['Market_Direction'] == 'DOWN']['Daily Return']

    if len(up_returns) >= 6:
        up_avg = up_returns.mean()
        down_avg = down_returns.mean()

        if down_avg < up_avg:
            results.append((data.index[i], data.index[i+9]))
# Displaying the date windows in which even with P(UP)>=0.6 we can lose money
for start, end in results:
    print(f"Window: {start.date()} to {end.date()}")

Window: 2024-04-22 to 2024-05-06
Window: 2024-04-23 to 2024-05-07
Window: 2024-05-31 to 2024-06-13
Window: 2024-06-03 to 2024-06-14
Window: 2024-06-13 to 2024-06-27
Window: 2024-06-18 to 2024-07-01
Window: 2024-06-19 to 2024-07-02
Window: 2024-06-20 to 2024-07-03
Window: 2024-06-21 to 2024-07-04
Window: 2024-06-24 to 2024-07-05
Window: 2024-06-25 to 2024-07-08
Window: 2024-07-10 to 2024-07-24
Window: 2024-07-11 to 2024-07-25
Window: 2024-07-12 to 2024-07-26
Window: 2024-07-15 to 2024-07-29
Window: 2024-07-16 to 2024-07-30
Window: 2024-07-18 to 2024-07-31
Window: 2024-07-19 to 2024-08-01
Window: 2024-07-22 to 2024-08-02
Window: 2024-07-23 to 2024-08-05
Window: 2024-07-24 to 2024-08-06
Window: 2024-07-25 to 2024-08-07
Window: 2024-07-31 to 2024-08-13
Window: 2024-08-01 to 2024-08-14
Window: 2024-08-02 to 2024-08-16
Window: 2024-08-05 to 2024-08-19
Window: 2024-08-06 to 2024-08-20
Window: 2024-08-07 to 2024-08-21
Window: 2024-08-08 to 2024-08-22
Window: 2024-08-09 to 2024-08-23
Window: 20


**Part 3**

Let's break down the profitability of this bet. Your **net return** is determined by the potential winnings from "up" days balanced against losses from "down" days.

The formula for net return is:
$$( \text{winnings per UP day} \times P(\text{UP}) ) - ( \text{losses per DOWN day} \times P(\text{DOWN}) ) = \text{Net Return}$$

Given the bet terms:

You win 100 dollar when your stock goes up.
You lose 150 dollar when your stock goes down
We also know that $P(\text{DOWN}) = 1 - P(\text{UP})$.

Substituting these values into the formula:
$$\text{Net Return} = (100 \times P(\text{UP})) - (150 \times (1 - P(\text{UP})))$$
$$\text{Net Return} = 100 \times P(\text{UP}) - 150 + 150 \times P(\text{UP})$$
$$\text{Net Return} = 250 \times P(\text{UP}) - 150$$

For this bet to be profitable, your **net return must be positive**:
$$250 \times P(\text{UP}) - 150 > 0$$
$$250 \times P(\text{UP}) > 150$$
$$P(\text{UP}) > \frac{150}{250}$$
$$P(\text{UP}) > 0.6$$

